# Bedrock Python Tutorial

This notebook is a tutorial on how to use Bedrock and its Python binding to configure and launch Mochi daemons.

## Bedrock: the basics

Bedrock is a generic daemon program that can load a JSON configuration representing the composition and configuration of a Mochi microservice, and setup the microservice accordingly.
The following command shows how to start an "empty" bedrock daemon using the "na+sm" (shared memory) tranport.

In [1]:
# Kill this process by clicking the square button (interrupt the kernel) at the top of the notebook
!bedrock na+sm

[2024-02-05 17:01:32.712] [info] Bedrock daemon now running at na+sm://12128-0
^C


More information about the parameters can be obtained as follows.

In [2]:
!bedrock --help


USAGE: 

   bedrock  [--jx9-context <x=1,y=2,z=something,...>] [-j] [--stdin] [-o
            <config-file>] [-c <config-file>] [-v <level>] [--] [--version]
            [-h] <address>


Where: 

   --jx9-context <x=1,y=2,z=something,...>
     Comma-separated list of Jx9 parameters for the Jx9 script

   -j,  --jx9
     Interpret configuration as a Jx9 script

   --stdin
     Read configuration from standard input

   -o <config-file>,  --output-config <config-file>
     JSON file to write after deployment

   -c <config-file>,  --config <config-file>
     JSON or JX9 configuration file

   -v <level>,  --verbose <level>
     Log level (trace, debug, info, warning, error, critical, off)

   --,  --ignore_rest
     Ignores the rest of the labeled arguments following this flag.

   --version
     Displays version information and exits.

   -h,  --help
     Displays usage information and exits.

   <address>
     (required)  Protocol (e.g. ofi+tcp) or address (e.g.
     ofi+tcp://127.0.0

Let's ask bedrock to output its initial configuration to a file.

In [3]:
# Kill this process by clicking the square button (interrupt the kernel) at the top of the notebook
!bedrock na+sm -o config.json

[2024-02-05 17:01:33.768] [info] Bedrock daemon now running at na+sm://12132-0
^C


In [4]:
import json
with open("config.json") as f:
    print(json.dumps(json.load(f), indent=4))

{
    "abt_io": [],
    "bedrock": {
        "pool": "__primary__",
        "provider_id": 0
    },
    "clients": [],
    "libraries": {},
    "margo": {
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "lazy_stack_alloc": false,
            "pools": [
                {
                    "access": "mpmc",
                    "kind": "fifo_wait",
                    "name": "__primary__"
                }
            ],
            "profiling_dir": ".",
            "xstreams": [
                {
                    "name": "__primary__",
                    "scheduler": {
                        "pools": [
                            0
                        ],
                        "type": "basic_wait"
                    }
                }
            ]
        },
        "enable_abt_profiling": false,
        "handle_cache_size": 32,
        "mercury": {
            "address": "na+sm://12132-0",
          

A bedrock configuration contains the following sections.
* `margo`: the configuration of the Margo instance, including its Argobots and Mercury configurations;
* `libraries`: a dictionary mapping bedrock module names to the path of the library to load;
* `bedrock`: the bedrock server configuration;
* `abt_io`: a list of ABT-IO instances;
* `ssg`: a list of SSG groups;
* `mona`: a list of MoNA instances;
* `providers`: a list of providers;
* `clients`: a list of clients;

The `libraries`, `abt_io`, `ssg`, `mona`, `providers`, and `clients` sections are empty since we haven't provided components yet.

This configuration can be a good starting point to start filling the bedrock daemon with actual components. Rather than writing JSON directly, however, it is recommended to use the `mochi.bedrock.spec` package to *programmatically* generate a configuration. This can be done in three different ways.
1. By using the `mochi.bedrock.spec` package to write the configuration, then by generating the corresponding JSON file and providing it as argument to the `bedrock` programm;
2. By using the `mochi.bedrock.spec` package in conjunction with the `mochi.bedrock.server` package to configure and start the server as a single Python program;
3. By starting an empty `bedrock` daemon, then using the `mochi.bedrock.spec` package in conjunction with the `mochi.bedrock.client` package to access the daemon remotely and reconfigure it at run time.

The following will explore these three options.

## Configuring bedrock using the mochi.bedrock.spec package

### Getting started

The root of a Bedrock configuration is a `ProcSpec`, defined in the `mochi.bedrock.spec` package. It defines everything we want to run on a process.

In [5]:
import mochi.bedrock.spec as spec

We can create a `ProcSpec` using a single `margo` key specifying the protocol we wish to use.

In [6]:
my_process = spec.ProcSpec(margo='na+sm')

All the specification objects have a `to_json()` method to generate their JSON configuration. Let's look at our proc's configuration:

In [7]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_abt_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "version": "unknown",
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "lazy_stack_alloc": false,
            "profiling_dir": ".",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler": {
                        "type": "basic_wait",
                        "pools": [
                            "__primary__"
                        ]
 

The classes in Bedrock's `spec` package give us a very simple way of accessing sub-configurations. Objects that have a name can be accessed using their name or their index in the list that contains them. For example, accessing Argobots' primary pool:

In [8]:
print(my_process.margo.argobots.pools['__primary__'])
print(my_process.margo.argobots.pools[0])

PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc')
PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc')


Let's customize our process a little more. What about defining a pool for RPCs to run on, along with a few execution streams using that pool?

In [9]:
# Create the new pool, it will be added automatically to the process
my_rpc_pool = my_process.margo.argobots.pools.add(name='my_rpc_pool', kind='fifo', access='mpmc')
# Create two execution streams using that pool
for i in range(0,4):
    sched = spec.SchedulerSpec(type='basic', pools=[my_rpc_pool])
    my_process.margo.argobots.xstreams.add(name=f'my_xstream_{i}', scheduler=sched)
# Now let's set the pools we want for handling RPCs
my_process.margo.rpc_pool = my_rpc_pool

Let's now print the resulting configuration:

In [10]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_abt_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "version": "unknown",
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "lazy_stack_alloc": false,
            "profiling_dir": ".",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler":

### Adding ABT-IO instances

Let's add an ABT-IO instance. To do so, we need to give the instance a name, a pool in which it will submit its 
ULTs, and a configuration (dictionary).

In [11]:
my_process.abt_io.add(name='my_abtio', pool=my_rpc_pool, config={})

AbtIOSpec(name='my_abtio', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'), config={})

### Adding SSG groups

Let's add and SSG group.

In [12]:
swim_config = spec.SwimSpec(
    period_length_ms=1000,
    suspect_timeout_periods=3,
    subgroup_member_count=0,
    disabled=False)

my_process.ssg.add(name='my_group',
                   swim=swim_config,
                   bootstrap='init',
                   group_file='my_group.ssg',
                   pool=my_rpc_pool)

SSGSpec(name='my_group', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'), credential=-1, bootstrap='init', group_file='my_group.ssg', swim=SwimSpec(period_length_ms=1000, suspect_timeout_periods=3, subgroup_member_count=0, disabled=False))

In [13]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_abt_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "version": "unknown",
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "lazy_stack_alloc": false,
            "profiling_dir": ".",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler":

### Adding MoNA instances

Let's add a MoNA instance.

In [14]:
my_process.mona.add(name='my_mona', pool=my_rpc_pool, config={})

MonaSpec(name='my_mona', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'), config={})

### Adding clients

Let's add some module clients. First we need to add module libraries.

In [15]:
# This requires Bedrock to have been compiled with -DENABLE_EXAMPLES=ON,
# please change the paths bellow to where the modules are actually located in your build
my_process.libraries['module_a'] = '/home/mdorier/Code/mochi-bedrock/build/examples/libexample-module-a.so'
my_process.libraries['module_b'] = '/home/mdorier/Code/mochi-bedrock/build/examples/libexample-module-b.so'

We can now add a couple of clients.

In [16]:
my_process.clients.add(
    name='ClientA',
    type='module_a',
    config={},
    dependencies={},
    tags=[])

my_process.clients.add(
    name='ClientB',
    type='module_b',
    config={},
    dependencies={},
    tags=[])

ClientSpec(name='ClientB', type='module_b', config={}, dependencies={}, tags=[])

### Adding providers

Now let's add providers. Bedrock has no idea what dependencies are required by providers of each module, so you need to make sure the dependencies and configurations are correct on your own.

In [17]:
my_process.providers.add(
    name='ProviderA',
    type='module_a',
    provider_id=42,
    pool=my_rpc_pool,
    config={},
    dependencies={},
    tags=[])

my_process.providers.add(
    name='ProviderB',
    type='module_b',
    provider_id=33,
    pool=my_rpc_pool,
    config={},
    dependencies={
        "ssg_group" : 'my_group',
        "a_provider" : "module_a:42",
        "a_provider_handle" : [ "ProviderA@ssg://my_group/0" ],
        "a_client" : "ClientA",
        "mona_instance": "my_mona"
    },
    tags=[])

ProviderSpec(name='ProviderB', type='module_b', pool=PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc'), provider_id=33, config={}, dependencies={'ssg_group': 'my_group', 'a_provider': 'module_a:42', 'a_provider_handle': ['ProviderA@ssg://my_group/0'], 'a_client': 'ClientA', 'mona_instance': 'my_mona'}, tags=[])

In [18]:
print(my_process.to_json(indent=4))

{
    "margo": {
        "progress_timeout_ub_msec": 100,
        "enable_abt_profiling": false,
        "enable_diagnostics": false,
        "handle_cache_size": 32,
        "profile_sparkline_timeslice_msec": 1000,
        "version": "unknown",
        "argobots": {
            "abt_mem_max_num_stacks": 8,
            "abt_thread_stacksize": 2097152,
            "version": "unknown",
            "lazy_stack_alloc": false,
            "profiling_dir": ".",
            "pools": [
                {
                    "name": "__primary__",
                    "kind": "fifo_wait",
                    "access": "mpmc"
                },
                {
                    "name": "my_rpc_pool",
                    "kind": "fifo",
                    "access": "mpmc"
                }
            ],
            "xstreams": [
                {
                    "name": "__primary__",
                    "cpubind": -1,
                    "affinity": [],
                    "scheduler":

### Deploying the service

Once you have programmatically added everything you needed in your process specification `my_process`, you can convert it to JSON and write it to a file:

In [19]:
with open("my_config.json", "w+") as f:
    f.write(my_process.to_json())

!cat my_config.json

{"margo": {"progress_timeout_ub_msec": 100, "enable_abt_profiling": false, "enable_diagnostics": false, "handle_cache_size": 32, "profile_sparkline_timeslice_msec": 1000, "version": "unknown", "argobots": {"abt_mem_max_num_stacks": 8, "abt_thread_stacksize": 2097152, "version": "unknown", "lazy_stack_alloc": false, "profiling_dir": ".", "pools": [{"name": "__primary__", "kind": "fifo_wait", "access": "mpmc"}, {"name": "my_rpc_pool", "kind": "fifo", "access": "mpmc"}], "xstreams": [{"name": "__primary__", "cpubind": -1, "affinity": [], "scheduler": {"type": "basic_wait", "pools": ["__primary__"]}}, {"name": "my_xstream_0", "cpubind": -1, "affinity": [], "scheduler": {"type": "basic", "pools": ["my_rpc_pool"]}}, {"name": "my_xstream_1", "cpubind": -1, "affinity": [], "scheduler": {"type": "basic", "pools": ["my_rpc_pool"]}}, {"name": "my_xstream_2", "cpubind": -1, "affinity": [], "scheduler": {"type": "basic", "pools": ["my_rpc_pool"]}}, {"name": "my_xstream_3", "cpubind": -1, "affinity"

You can now pass this file to the `bedrock` command to start your process:

In [20]:
!bedrock na+sm -c my_config.json

Registered a client from module A
 -> mid = 0x5556661fdc30
Initializing client from module B
Registered a provider from module A
 -> mid         = 0x5556661fdc30
 -> provider id = 42
 -> pool        = 0x5556661f7d80
 -> config      = {}
 -> name        = ProviderA
Created provider handle from module A
Registering a provider from module B
 -> mid         = 0x5556661fdc30
 -> provider_id = 33
 -> pool        = 0x5556661f7d80
 -> config      = {}
 -> name        = ProviderB
[2024-02-05 17:01:35.956] [info] Bedrock daemon now running at na+sm://12136-0
^C


## Starting bedrock from python using mochi.bedrock.server

Having seen how to programmatically generate a Bedrock configuration, we can go one step further and actually use Python to start the server instance.

In [21]:
import mochi.bedrock.server as server

daemon = server.Server.from_spec(my_process)

Registered a client from module A
 -> mid = 0x555689575ce0
Initializing client from module B
Registered a provider from module A
 -> mid         = 0x555689575ce0
 -> provider id = 42
 -> pool        = 0x555689571180
 -> config      = {}
 -> name        = ProviderA
Created provider handle from module A
Registering a provider from module B
 -> mid         = 0x555689575ce0
 -> provider_id = 33
 -> pool        = 0x555689571180
 -> config      = {}
 -> name        = ProviderB
[2024-02-05 17:01:38.291] [info] Bedrock daemon now running at na+sm://12111-0


Now our Python process runs a Bedrock daemon. We can access its runtime configuration and even convert it back to a spec:

In [22]:
print(daemon.config)

{'abt_io': [{'config': {'internal_pool_flag': False, 'null_io_read': False, 'null_io_write': False, 'trace_io': False, 'version': '0.6.0'}, 'name': 'my_abtio', 'pool': 'my_rpc_pool'}], 'bedrock': {'pool': '__primary__', 'provider_id': 0}, 'clients': [{'config': {}, 'dependencies': {}, 'name': 'ClientA', 'tags': [], 'type': 'module_a'}, {'config': {}, 'dependencies': {}, 'name': 'ClientB', 'tags': [], 'type': 'module_b'}], 'libraries': {'module_a': '/home/mdorier/Code/mochi-bedrock/build/examples/libexample-module-a.so', 'module_b': '/home/mdorier/Code/mochi-bedrock/build/examples/libexample-module-b.so'}, 'margo': {'argobots': {'abt_mem_max_num_stacks': 8, 'abt_thread_stacksize': 2097152, 'lazy_stack_alloc': False, 'pools': [{'access': 'mpmc', 'kind': 'fifo_wait', 'name': '__primary__'}, {'access': 'mpmc', 'kind': 'fifo', 'name': 'my_rpc_pool'}], 'profiling_dir': '.', 'xstreams': [{'name': '__primary__', 'scheduler': {'pools': [0], 'type': 'basic_wait'}}, {'name': 'my_xstream_0', 'sche

In [23]:
print(daemon.spec)

ProcSpec(margo=MargoSpec(mercury=MercurySpec(address='na+sm://12111-0', listening=True, ip_subnet='', auth_key='', auto_sm=False, max_contexts=1, na_no_block=False, na_no_retry=False, no_bulk_eager=False, no_loopback=False, request_post_incr=256, request_post_init=256, stats=False, version='2.3.1', checksum_level='none', input_eager_size=4080, output_eager_size=4080, na_addr_format='unspec', na_max_expected_size=0, na_max_unexpected_size=0, na_request_mem_device=False), argobots=ArgobotsSpec(abt_mem_max_num_stacks=8, abt_thread_stacksize=2097152, version='unknown', _pools=[PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc'), PoolSpec(name='my_rpc_pool', kind='fifo', access='mpmc')], _xstreams=[XstreamSpec(name='__primary__', scheduler=SchedulerSpec(type='basic_wait', pools=[PoolSpec(name='__primary__', kind='fifo_wait', access='mpmc')]), cpubind=-1, affinity=[]), XstreamSpec(name='my_xstream_0', scheduler=SchedulerSpec(type='basic', pools=[PoolSpec(name='my_rpc_pool', kind='f

You can manipulate the runtime configuration of your server. For instance, here is how to add a new Argobots pool and an execution stream that uses it.

In [24]:
daemon.margo.pools.create(spec.PoolSpec(name="new_pool", kind="fifo_wait", access="mpmc"))
daemon.margo.xstreams.create(
    spec.XstreamSpec(
        name="new_xstream",
        scheduler=spec.SchedulerSpec(
            type="basic_wait",
            pools=[daemon.margo.spec.argobots.pools["new_pool"]])))

Finally, to block the process and keep running the daemon, use `daemon.wait_for_finalize()`. To finalize it, use `daemon.finalize()`.

In [25]:
daemon.finalize()

Destroyed provider handle from module A
Deregistered a provider from module A
Deregistring provider from module B
Finalized a client from module A
Finalizing client from module B
